In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
    dff = pd.read_csv('C:/Users/Don/Desktop/FiresNEW.csv', encoding = 'ISO-8859-1', low_memory=False)
    dff = (
        dff.drop(columns=["FOD_ID","FPA_ID","SOURCE_SYSTEM_TYPE","SOURCE_SYSTEM","NWCG_REPORTING_AGENCY","NWCG_REPORTING_UNIT_ID","NWCG_REPORTING_UNIT_NAME","SOURCE_REPORTING_UNIT","SOURCE_REPORTING_UNIT_NAME","LOCAL_FIRE_REPORT_ID","LOCAL_INCIDENT_ID","FIRE_CODE","FIRE_NAME","ICS_209_PLUS_INCIDENT_JOIN_ID","ICS_209_PLUS_COMPLEX_JOIN_ID","MTBS_ID","MTBS_FIRE_NAME","COMPLEX_NAME","DISCOVERY_DATE","DISCOVERY_DOY","DISCOVERY_TIME","NWCG_GENERAL_CAUSE","NWCG_CAUSE_AGE_CATEGORY","CONT_DATE","CONT_DOY","CONT_TIME","LATITUDE","LONGITUDE","OWNER_DESCR","COUNTY","FIPS_CODE","FIPS_NAME"])
        .drop(dff[dff.NWCG_CAUSE_CLASSIFICATION == 'Human'].index)
        .drop(dff[dff.NWCG_CAUSE_CLASSIFICATION == 'Missing data/not specified/undetermined'].index)
        .dropna()
        .drop(columns=['NWCG_CAUSE_CLASSIFICATION'])
    )
dff = dff.drop(dff[(dff['FIRE_SIZE_CLASS'] == 'A')].index)
dff.head()


,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,STATE
24,2004,2.5,B,NM
25,2004,10.0,C,NM
47,2005,0.5,B,CO
54,2005,0.3,B,WA
56,2005,2.0,B,OR


In [19]:
    dfstate = pd.read_csv('C:/Users/Don/Desktop/csvData.csv')
    dfstate = (
        dfstate.drop(columns='Abbrev')
        .rename(columns={'Code':'STATE'})
    )
    
    
   

In [17]:
    df3 = (
        dff.set_index('STATE').combine_first(dfstate.set_index('STATE'))
        .reset_index()
        .drop(columns='STATE')
        .dropna()
        .rename(columns={'FIRE_YEAR':'Date','State':'Location'})
    )
    df3['Date'] = df3['Date'].astype(int)
df3

,FIRE_SIZE,FIRE_SIZE_CLASS,Date,Location
0,10.0,C,2009,Alaska
1,1.0,B,1993,Alaska
2,1.0,B,1993,Alaska
3,0.5,B,1997,Alaska
4,4.0,B,1999,Alaska
...,...,...,...,...
142214,500.0,E,2018,Wyoming
142215,4.0,B,2018,Wyoming
142216,100.0,D,2018,Wyoming
142217,1.0,B,2018,Wyoming


In [24]:
dftemp=pd.read_csv('C:/Users/Don/Desktop/tavg.csv')
f = lambda dftemp : dftemp['Location'].split("CD")
dftemp['Location'] = dftemp.apply(f, axis=1)
g = lambda dftemp : dftemp['Location'].pop(0)
dftemp['Location'] = dftemp.apply(g, axis=1)
dftemp.reset_index()
dftemp = dftemp.drop(columns=['Location ID','Rank','Anomaly (1901-2000 base period)','1901-2000 Mean'])
dftemp['Location'] = dftemp['Location'].str.rstrip()
dftemp['Celsius'] = (dftemp['Value']-32)*5/9
dftemp['Celsius'] = dftemp['Celsius'].round(1)
dftemp = dftemp.drop(columns=['Value'])
dftemp = dftemp.drop(dftemp[dftemp['Date'] % 10 != 7].index)
dftemp['Date'] = dftemp['Date']//100
dftemp['Date'] = dftemp['Date'].astype(int)
dftemp = dftemp.drop(dftemp[dftemp.Date > 2018].index)
dftemp = dftemp.drop(dftemp[dftemp['Date'] < 1992].index)
dftemp = dftemp.reset_index(drop = True)
dftemp = dftemp.reset_index(drop = False)



dffinal = (
    df3.groupby(['Date','Location']).size()
    .to_frame()
    .reset_index()
)
df3.drop(df3[df3.Location == 'Alaska'].index)
df3.drop(df3[df3.Location == 'Hawaii'].index)
dffinal = (    
    dffinal.sort_values(by=['Location','Date'])
    .rename(columns={0: '# of Fires'})
    .reset_index(drop=True)
     .reset_index(drop=False)
    )

d1=dffinal
d1 = d1.drop(columns=['index'])
d1['Date']=d1['Date'].values.astype(str)
d1['Location']=d1['Location'].values.astype(str)
d1['id'] = d1['Date'].str.cat(d1['Location'],sep='-')
d1


dftemp12 = dftemp
dftemp12 = dftemp12.drop(columns=['index'])
dftemp12['Date']=dftemp12['Date'].values.astype(str)
dftemp12['Location']=dftemp12['Location'].values.astype(str)
dftemp12['id'] = dftemp12['Date'].str.cat(dftemp12['Location'],sep='-')
dftemp100 = dftemp12.drop(columns=['Date','Location'])
dftemp100

dffinal = pd.merge(d1,dftemp100,how='right',on='id')
dffinal = dffinal.drop(columns=['id'])

dffinal.head()

,Date,Location,# of Fires,Celsius
0,1992,Alabama,2.0,25.6
1,1993,Alabama,7.0,27.6
2,1994,Alabama,11.0,24.7
3,1995,Alabama,55.0,26.1
4,1996,Alabama,32.0,25.4
